In [24]:
import datetime
from pathlib import Path
from typing import cast

import polars as pl
from scipy.optimize import newton

pl.Config.set_tbl_hide_dataframe_shape(True)

polars.config.Config

In [30]:
def get_data(
    source: str | Path,
) -> pl.DataFrame:
    return cast(
        pl.DataFrame,
        pl.read_excel(
            source=source,
        ).with_columns(pl.col("date").str.to_date()),
    )


def bond_duration_convexity(row: dict) -> dict:
    coupon = row["coupon rate"]
    price = row["clean price"]
    ytm = row["ytm"]
    maturity_date = row["maturity date"]

    today = datetime.date(2023, 12, 29)
    years_to_maturity = (maturity_date - today).days / 365
    periods = int(years_to_maturity * 2)
    semi_annual_coupon = (coupon / 2) * 100
    semi_annual_ytm = ytm / 2

    pv_cf = sum(
        (semi_annual_coupon / (1 + semi_annual_ytm) ** t)
        for t in range(1, periods)
    )
    pv_cf += (semi_annual_coupon + 100) / (1 + semi_annual_ytm) ** periods

    weighted_time = sum(
        (t / 2) * (semi_annual_coupon / (1 + semi_annual_ytm) ** t)
        for t in range(1, periods)
    )
    weighted_time += (
        (periods / 2)
        * (semi_annual_coupon + 100)
        / (1 + semi_annual_ytm) ** periods
    )

    macaulay_duration = weighted_time / price
    modified_duration = macaulay_duration / (1 + semi_annual_ytm)

    weighted_time_sq = sum(
        (t / 2) ** 2 * (semi_annual_coupon / (1 + semi_annual_ytm) ** t)
        for t in range(1, periods)
    )
    weighted_time_sq += (
        (periods / 2) ** 2
        * (semi_annual_coupon + 100)
        / (1 + semi_annual_ytm) ** periods
    )

    convexity = (weighted_time_sq / price) / (1 + semi_annual_ytm) ** 2

    return {
        "modified_duration": modified_duration,
        "macaulay_duration": macaulay_duration,
        "convexity": convexity,
    }


def bond_price(ytm, coupon, years, par=100):
    periods = int(years * 2)
    semi_annual_coupon = (coupon / 2) * par
    semi_annual_ytm = ytm / 2
    return (
        sum(
            semi_annual_coupon / (1 + semi_annual_ytm) ** t
            for t in range(1, periods + 1)
        )
        + par / (1 + semi_annual_ytm) ** periods
    )


def price_diff(ytm, coupon, price, years):
    return bond_price(ytm, coupon, years) - price

In [22]:
data = (
    pl.DataFrame(
        {
            "metric": [
                "issue date",
                "maturity date",
                "coupon rate",
                "clean price",
                "accrued interest",
                "ytm",
            ],
            "207391": [
                "2019-08-15",
                "2029-08-15",
                "0.01625",
                "89.03125",
                "0.6005",
                "0.037677",
            ],
            "204095": [
                "1999-08-15",
                "2029-08-15",
                "0.06125",
                "111.0391",
                "2.2636",
                "0.038784",
            ],
        }
    )
    .unpivot(
        index="metric",
        variable_name="bond_id",
        value_name="value",
    )
    .pivot(
        on="metric",
        index="bond_id",
        values="value",
    )
).with_columns(
    pl.col("ytm").cast(pl.Float64),
    pl.col("accrued interest").cast(pl.Float64),
    pl.col("clean price").cast(pl.Float64),
    pl.col("coupon rate").cast(pl.Float64),
    pl.col("issue date").cast(pl.Date),
    pl.col("maturity date").cast(pl.Date),
)
data

bond_id,issue date,maturity date,coupon rate,clean price,accrued interest,ytm
str,date,date,f64,f64,f64,f64
"""207391""",2019-08-15,2029-08-15,0.01625,89.03125,0.6005,0.037677
"""204095""",1999-08-15,2029-08-15,0.06125,111.0391,2.2636,0.038784


# 1.1

In [5]:
ytm_spread = (
    data.select("bond_id", "ytm")
    .with_columns((pl.col("ytm").cast(pl.Float64) * 10000).alias("ytm_bps"))
    .sort("ytm_bps", descending=True)
)

price_analysis = data.select("bond_id", "clean price").with_columns(
    (pl.col("clean price").cast(pl.Float64) - 100).alias("price_vs_par")
)

print(f"""YTM Spread (bps):"
{ytm_spread}
Price vs Par:
{price_analysis}""")

YTM Spread (bps):"
┌─────────┬──────────┬─────────┐
│ bond_id ┆ ytm      ┆ ytm_bps │
│ ---     ┆ ---      ┆ ---     │
│ str     ┆ f64      ┆ f64     │
╞═════════╪══════════╪═════════╡
│ 204095  ┆ 0.038784 ┆ 387.84  │
│ 207391  ┆ 0.037677 ┆ 376.77  │
└─────────┴──────────┴─────────┘
Price vs Par:
┌─────────┬─────────────┬──────────────┐
│ bond_id ┆ clean price ┆ price_vs_par │
│ ---     ┆ ---         ┆ ---          │
│ str     ┆ f64         ┆ f64          │
╞═════════╪═════════════╪══════════════╡
│ 207391  ┆ 89.03125    ┆ -10.96875    │
│ 204095  ┆ 111.0391    ┆ 11.0391      │
└─────────┴─────────────┴──────────────┘


You should go long 204095 as it yields 11 bps more with the same maturity and trades at a premium, suggesting the market sees it as higher quality. Short 207391; it's the riskier bond trading at a 11-point discount. If that risk premium compresses or the bonds converge in yield, you make money on both legs.

# 1.2
## Long Leg (204095)
- Use a reverse repo against the bond.
- Post the bond as collateral to a dealer or repo market, borrow cash at the repo rate, and use that cash to buy the bond.
- The carry is the bond's yield minus the repo rate
    - Given the 3.87% YTM versus, if repo rates are around 4%, you'd be paying a bit for carry, but you're betting on spread compression to offset it.

## Short Leg (207391)
- Short sell the bond at 89.03125.
- Thr P&L on the short benefits if the price falls, or if you cover at a lower price.

## Net financing
- The positions partially offset. 
- Only need to fund the ~23-point difference via repo or margin.
- Thr net carry is the yield pickup (11 bps) minus financing costs (repo spread), making it relatively low-cost to implement (if spreads tighten).

# 1.3
## Risks

### Repo funding squeeze
- If repo rates spike or the dealer demands higher haircuts, carry flips negative.
- You're borrowing at potentially rising rates to finance a long that yields a fixed 3.87%.

### Liquidity mismatch
- If you need to unwind quickly, bond 207391 (the discount bond) likely has worse liquidity.
- You might struggle to cover the short at reasonable prices.

### Curve/duration misalignment
- Even with identical maturities, they have different duration sensitivities, see below.
- Parallel yield curve moves hurt you asymmetrically.

### Arbitrage?
- Not a short-term arbitrage.
- True arbitrage is risk-free.
- This trade requires a bet that the market is mispricing relative risk.
    - You're exposed to the 11-point gap widening instead of narrowing.
    - You need carry and mean reversion to work in your favor.

In [20]:
results = []
for row in data.to_dicts():
    dur_conv = bond_duration_convexity(row)
    results.append(
        {
            "bond_id": row["bond_id"],
            **dur_conv,
        }
    )

metrics_df = pl.DataFrame(results)
print(metrics_df)

┌─────────┬───────────────────┬───────────────────┬───────────┐
│ bond_id ┆ modified_duration ┆ macaulay_duration ┆ convexity │
│ ---     ┆ ---               ┆ ---               ┆ ---       │
│ str     ┆ f64               ┆ f64               ┆ f64       │
╞═════════╪═══════════════════╪═══════════════════╪═══════════╡
│ 207391  ┆ 5.194564          ┆ 5.292422          ┆ 27.605021 │
│ 204095  ┆ 4.705245          ┆ 4.796489          ┆ 24.071712 │
└─────────┴───────────────────┴───────────────────┴───────────┘


# 1.4

In [29]:
ytm_new = []
for row in data.to_dicts():
    ytm = newton(
        lambda y: price_diff(y, row["coupon rate"], row["clean price"], 5.5),
        row["ytm"],
    )
    ytm_new.append(ytm)

bonds = data.with_columns(pl.Series("ytm_new", ytm_new))
print(bonds.select("bond_id", "clean price", "ytm", "ytm_new"))

┌─────────┬─────────────┬──────────┬──────────┐
│ bond_id ┆ clean price ┆ ytm      ┆ ytm_new  │
│ ---     ┆ ---         ┆ ---      ┆ ---      │
│ str     ┆ f64         ┆ f64      ┆ f64      │
╞═════════╪═════════════╪══════════╪══════════╡
│ 207391  ┆ 89.03125    ┆ 0.037677 ┆ 0.038575 │
│ 204095  ┆ 111.0391    ┆ 0.038784 ┆ 0.03877  │
└─────────┴─────────────┴──────────┴──────────┘


# 1.4

In [33]:
_data = (
    data.with_columns(
        [
            pl.lit(87.0).alias("price_20240215_207391"),
            pl.lit(113.0).alias("price_20240215_204095"),
            pl.lit(0.0465).alias("ytm_20240815_207391"),
            pl.lit(0.0470).alias("ytm_20240815_204095"),
        ]
    )
    .with_columns(
        [
            pl.when(pl.col("bond_id") == "207391")
            .then(bond_price(0.0465, 0.01625, 5.0))
            .otherwise(bond_price(0.0470, 0.06125, 5.0))
            .alias("clean_price_20240815"),
        ]
    )
    .with_columns(
        [
            (pl.col("clean price") + pl.col("accrued interest")).alias(
                "dirty_price_original"
            ),
            (pl.col("clean_price_20240815") + 0).alias(
                "dirty_price_20240815"
            ),
        ]
    )
)

print(_data.select("bond_id", "dirty_price_original", "dirty_price_20240815"))

┌─────────┬──────────────────────┬──────────────────────┐
│ bond_id ┆ dirty_price_original ┆ dirty_price_20240815 │
│ ---     ┆ ---                  ┆ ---                  │
│ str     ┆ f64                  ┆ f64                  │
╞═════════╪══════════════════════╪══════════════════════╡
│ 207391  ┆ 89.63175             ┆ 86.641993            │
│ 204095  ┆ 113.3027             ┆ 106.284458           │
└─────────┴──────────────────────┴──────────────────────┘


In [34]:
_data = data.with_columns(
    [
        pl.when(pl.col("bond_id") == "207391")
        .then(bond_price(0.0465, 0.01625, 5.0))
        .otherwise(bond_price(0.0470, 0.06125, 5.0))
        .alias("clean_price_081524"),
    ]
).with_columns(
    [
        (pl.col("clean price") + pl.col("accrued interest")).alias(
            "dirty_price_original"
        ),
        (pl.col("clean_price_081524") + 0).alias("dirty_price_081524"),
    ]
)

position_original = (
    _data.filter(pl.col("bond_id") == "204095")["dirty_price_original"][0]
    - _data.filter(pl.col("bond_id") == "207391")["dirty_price_original"][0]
)

position_081524 = (
    _data.filter(pl.col("bond_id") == "204095")["dirty_price_081524"][0]
    - _data.filter(pl.col("bond_id") == "207391")["dirty_price_081524"][0]
)

pnl = position_081524 - position_original

print(f"""Position value (original): {position_original:.2f}
Position value (2024-08-15): {position_081524:.2f}
P&L: {pnl:.2f}""")

Position value (original): 23.67
Position value (2024-08-15): 19.64
P&L: -4.03
